In [1]:
import itertools
import sys

sys.path.append("..")
import polars as pl

from src.utils.io import load_pickle

In [5]:
agent_feat = load_pickle("../data/feature/agent_parsed_feature.pkl")
utility_agent1 = load_pickle("../data/feature/utility_agent1.pkl")
fold = load_pickle("../data/feature/fold.pkl")

train = pl.read_csv("../data/raw/train.csv")

In [8]:
result_agent1 = train.select(
    pl.col(
        [
            "num_wins_agent1",
            "num_draws_agent1",
            "num_losses_agent1",
            "utility_agent1",
        ]
    )
)

In [21]:
win_rate = result_agent1.select(
    (
        pl.col("num_wins_agent1")
        / (
            pl.col("num_wins_agent1")
            + pl.col("num_draws_agent1")
            + pl.col("num_losses_agent1")
        )
    ).alias("win_rate_agent1")
)

In [22]:
agent_feat.select(pl.all().n_unique())

agent1_selection,agent1_expansion,agent1_playout,agent1_score_bounds,agent2_selection,agent2_expansion,agent2_playout,agent2_score_bounds
u32,u32,u32,u32,u32,u32,u32,u32
4,3,3,2,4,3,3,2


In [23]:
agent_feat.columns

['agent1_selection',
 'agent1_expansion',
 'agent1_playout',
 'agent1_score_bounds',
 'agent2_selection',
 'agent2_expansion',
 'agent2_playout',
 'agent2_score_bounds']

In [39]:
cols = [
    "agent1_expansion",
    "agent1_playout",
    "agent1_score_bounds",
    "agent1_selection",
    "agent2_expansion",
    "agent2_playout",
    "agent2_score_bounds",
    "agent2_selection",
]

cols_group = []
cols_group += cols
cols_group += list(itertools.combinations(cols, 2))

data: pl.DataFrame = pl.concat(
    [agent_feat, fold, utility_agent1, win_rate], how="horizontal"
)


cols_select = []
for col_group in cols_group:
    if isinstance(col_group, str):
        col_group = [col_group]
    elif isinstance(col_group, tuple):
        col_group = list(col_group)

    data = data.with_columns(
        pl.when(pl.col("utility_agent1") == 1.0)
        .then(1)
        .otherwise(0)
        .mean()
        .over(["fold"] + col_group)
        .alias(
            "{}_te_one_mean".format("_".join(col_group)),
        ),
        pl.when(pl.col("utility_agent1") == 0.0)
        .then(1)
        .otherwise(0)
        .mean()
        .over(["fold"] + col_group)
        .alias(
            "{}_te_zero_mean".format("_".join(col_group)),
        ),
        pl.when(pl.col("utility_agent1").abs() < 1.0)
        .then(pl.col("utility_agent1"))
        .otherwise(None)
        .mean()
        .over(["fold"] + col_group)
        .alias(
            "{}_te_non_one_mean".format("_".join(col_group)),
        ),
    )
    cols_select.append("{}_te_one_mean".format("_".join(col_group)))
    cols_select.append("{}_te_zero_mean".format("_".join(col_group)))
    cols_select.append("{}_te_non_one_mean".format("_".join(col_group)))

In [40]:
data.select(cols_select)

agent1_expansion_te_one_mean,agent1_expansion_te_zero_mean,agent1_expansion_te_non_one_mean,agent1_playout_te_one_mean,agent1_playout_te_zero_mean,agent1_playout_te_non_one_mean,agent1_score_bounds_te_one_mean,agent1_score_bounds_te_zero_mean,agent1_score_bounds_te_non_one_mean,agent1_selection_te_one_mean,agent1_selection_te_zero_mean,agent1_selection_te_non_one_mean,agent2_expansion_te_one_mean,agent2_expansion_te_zero_mean,agent2_expansion_te_non_one_mean,agent2_playout_te_one_mean,agent2_playout_te_zero_mean,agent2_playout_te_non_one_mean,agent2_score_bounds_te_one_mean,agent2_score_bounds_te_zero_mean,agent2_score_bounds_te_non_one_mean,agent2_selection_te_one_mean,agent2_selection_te_zero_mean,agent2_selection_te_non_one_mean,agent1_expansion_agent1_playout_te_one_mean,agent1_expansion_agent1_playout_te_zero_mean,agent1_expansion_agent1_playout_te_non_one_mean,agent1_expansion_agent1_score_bounds_te_one_mean,agent1_expansion_agent1_score_bounds_te_zero_mean,agent1_expansion_agent1_score_bounds_te_non_one_mean,agent1_expansion_agent1_selection_te_one_mean,agent1_expansion_agent1_selection_te_zero_mean,agent1_expansion_agent1_selection_te_non_one_mean,agent1_expansion_agent2_expansion_te_one_mean,agent1_expansion_agent2_expansion_te_zero_mean,agent1_expansion_agent2_expansion_te_non_one_mean,agent1_expansion_agent2_playout_te_one_mean,…,agent1_score_bounds_agent2_playout_te_non_one_mean,agent1_score_bounds_agent2_score_bounds_te_one_mean,agent1_score_bounds_agent2_score_bounds_te_zero_mean,agent1_score_bounds_agent2_score_bounds_te_non_one_mean,agent1_score_bounds_agent2_selection_te_one_mean,agent1_score_bounds_agent2_selection_te_zero_mean,agent1_score_bounds_agent2_selection_te_non_one_mean,agent1_selection_agent2_expansion_te_one_mean,agent1_selection_agent2_expansion_te_zero_mean,agent1_selection_agent2_expansion_te_non_one_mean,agent1_selection_agent2_playout_te_one_mean,agent1_selection_agent2_playout_te_zero_mean,agent1_selection_agent2_playout_te_non_one_mean,agent1_selection_agent2_score_bounds_te_one_mean,agent1_selection_agent2_score_bounds_te_zero_mean,agent1_selection_agent2_score_bounds_te_non_one_mean,agent1_selection_agent2_selection_te_one_mean,agent1_selection_agent2_selection_te_zero_mean,agent1_selection_agent2_selection_te_non_one_mean,agent2_expansion_agent2_playout_te_one_mean,agent2_expansion_agent2_playout_te_zero_mean,agent2_expansion_agent2_playout_te_non_one_mean,agent2_expansion_agent2_score_bounds_te_one_mean,agent2_expansion_agent2_score_bounds_te_zero_mean,agent2_expansion_agent2_score_bounds_te_non_one_mean,agent2_expansion_agent2_selection_te_one_mean,agent2_expansion_agent2_selection_te_zero_mean,agent2_expansion_agent2_selection_te_non_one_mean,agent2_playout_agent2_score_bounds_te_one_mean,agent2_playout_agent2_score_bounds_te_zero_mean,agent2_playout_agent2_score_bounds_te_non_one_mean,agent2_playout_agent2_selection_te_one_mean,agent2_playout_agent2_selection_te_zero_mean,agent2_playout_agent2_selection_te_non_one_mean,agent2_score_bounds_agent2_selection_te_one_mean,agent2_score_bounds_agent2_selection_te_zero_mean,agent2_score_bounds_agent2_selection_te_non_one_mean
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.101151,0.071419,-0.025766,0.117436,0.071767,0.050825,0.113352,0.066959,0.030591,0.105785,0.069575,0.022433,0.117841,0.073465,0.011971,0.118927,0.07599,0.024257,0.116939,0.068075,0.03117,0.149496,0.065695,0.036192,0.098605,0.066883,-0.005193,0.09229,0.065333,-0.040499,0.077531,0.067075,-0.108489,0.095963,0.068165,-0.049992,0.099522,…,0.023784,0.100837,0.059353,0.030159,0.138026,0.059709,0.038673,0.100873,0.076232,-0.000525,0.108453,0.06736,0.01469,0.101294,0.066019,0.027066,0.111951,0.069834,0.035925,0.107904,0.07284,-0.00831,